In [2]:

import holoviews as hv
import panel as pn
import numpy as np
import pandas as pd
import geopandas as gpd
import re
from panel.template import FastGridTemplate
from panel.template import DarkTheme
import hvplot.pandas


MAIN = "#D6ADD0"
ACCENT = "#724175"


In [3]:
df = pd.read_csv("./data/trips_by_year.csv", sep=",")
df2 = pd.read_csv("./data/trips_by_month.csv", sep=",")
df3 = pd.read_csv("./data/trips_by_hour.csv", sep=",")
df3["mean"] = df3["trips"].mean()




In [14]:
#### working with kdims 
def _create_barandlinewidget(data,title, x, xlabel):
   
    def plot(column="duration(year)"):
        def hooks(plot, element):
            p = plot.state
            p.toolbar.autohide = True
            p.toolbar.logo = None
            if "trips" in column:
                p.yaxis.ticker=[0,2000000,4000000,6000000,8000000]
                p.yaxis.major_label_overrides={0:"0",2000000:"2M",4000000:"4M",6000000:"6M",8000000:"8M"} 
            else:
                p.yaxis.ticker = np.arange(0, 1400, 200)
                p.yaxis.major_label.overrides={0:"0", 200:"200", 400:"400", 600:"600", 800:"800", 1000:"1000",1200:"1200", 1400:"1400"}
    
        return data.hvplot(kind="bar",
                           x=x,
                           title=title,
                           y=column,
                           color=MAIN,
                           xlabel=xlabel,
                           line_color="white",
                           responsive=True,
                           min_width=350,
                           min_height=250,
                           ylabel=re.sub(r"\([a-z,A-Z]*\)","", str(column)).capitalize(),
                           label=re.sub(r"\([a-z,A-Z]*\)","", str(column)).capitalize()).opts(legend_position='top', hooks=[hooks]) \
                               * data.hvplot(kind="line",
                                           x=x,
                                           y=f"mean{column}",
                                           responsive=True, 
                                           label="Mean")

    dmap = hv.DynamicMap(plot, kdims=["column"],sizing_mode="stretch_both").redim.values(column=[f"duration({x})", f"trips({x})"])
    hv_panel = pn.panel(dmap)
    widget = hv_panel[1]
    return pn.Column(
        pn.Row(*widget, height=100, width=200),
        hv_panel[0],
        sizing_mode="stretch_both",
    )
    

In [5]:
"""def _otherwaytodoit(data,title, x, widget, xlabel):
    def hooks(plot, element):
        p = plot.state
        p.toolbar.autohide = True
        p.toolbar.logo = None
        if "trips" in widget.value:
            p.yaxis.ticker=[0,2000000,4000000,6000000,8000000]
            p.yaxis.major_label_overrides={0:"0",2000000:"2M",4000000:"4M",6000000:"6M",8000000:"8M"} 
        else:
            p.yaxis.ticker = np.arange(0, 1400, 200)
            p.yaxis.major_label.overrides={0:"0", 200:"200", 400:"400", 600:"600", 800:"800", 1000:"1000",1200:"1200", 1400:"1400"}
    
    def plot(widget = widget):
        return data.hvplot(kind="bar",
                           x=x,
                           title=title,
                           xlabel=xlabel,
                           ylabel=re.sub(r"\([a-z,A-Z]*\)","", str(widget)).capitalize(),
                           y=widget,
                           color=MAIN,
                           line_color="white",
                           responsive=True,
                           min_width=350,
                           min_height=250,
                           label=re.sub(r"\([a-z,A-Z]*\)","", str(widget)).capitalize()).opts(legend_position='top',hooks=[hooks]) \
            * data.hvplot.line(x=x,
                             y=f"mean{widget}",
                             color=ACCENT,
                             responsive=True,
                             line_color="white",
                             min_width=350,
                             min_height=250,
                             label="Mean")
    plot = pn.bind(plot, widget=widget)
    return pn.Column(
        pn.pane.HoloViews(hv.DynamicMap(plot), sizing_mode="stretch_both"),
                sizing_mode="stretch_both",
    )"""

'def _otherwaytodoit(data,title, x, widget, xlabel):\n    def hooks(plot, element):\n        p = plot.state\n        p.toolbar.autohide = True\n        p.toolbar.logo = None\n        if "trips" in widget.value:\n            p.yaxis.ticker=[0,2000000,4000000,6000000,8000000]\n            p.yaxis.major_label_overrides={0:"0",2000000:"2M",4000000:"4M",6000000:"6M",8000000:"8M"} \n        else:\n            p.yaxis.ticker = np.arange(0, 1400, 200)\n            p.yaxis.major_label.overrides={0:"0", 200:"200", 400:"400", 600:"600", 800:"800", 1000:"1000",1200:"1200", 1400:"1400"}\n    \n    def plot(widget = widget):\n        return data.hvplot(kind="bar",\n                           x=x,\n                           title=title,\n                           xlabel=xlabel,\n                           ylabel=re.sub(r"\\([a-z,A-Z]*\\)","", str(widget)).capitalize(),\n                           y=widget,\n                           color=MAIN,\n                           line_color="white",\n    

In [6]:
def _create_barandline(data,title, x, y, y2, xlabel, ylabel, label, label2):
    def hooks(plot, element):
        p = plot.state
        p.toolbar.autohide = True
        p.toolbar.logo = None
        p.yaxis.ticker=[0,2000000,4000000,6000000,8000000]
        p.yaxis.major_label_overrides={0:"0",2000000:"2M",4000000:"4M",6000000:"6M",8000000:"8M"} 
        p.toolbar_location = "above"
 
    def plot():
        return data.hvplot.bar(title=title,
                               x=x,
                               xlabel=xlabel,
                               ylabel=ylabel,
                               y=y,
                               color=MAIN,
                               responsive=True,
                               line_color="white",
                               min_width=350,
                               min_height=250,
                               label=label).opts(legend_position="right",
                               hooks=[hooks]) \
        * df3.hvplot.line(x=x,
                          y=y2,
                          color=ACCENT,
                          responsive=True,
                          min_width=300,
                          min_height=150,
                          line_color="white",
                          label=label2)
    return pn.Column(
        pn.pane.HoloViews(hv.DynamicMap(plot), sizing_mode="stretch_both"),
                sizing_mode="stretch_both",
    )

In [7]:
def _create_geocoord():
    docks = pd.read_csv("./data/coordinates.csv")
    docks["geometry"] = "POINT (" + docks["lon"].astype(str) + " " + docks["lat"].astype(str) + ")"
    docks = docks[["station", "geometry"]]
    docks.rename(columns={"station": "name"}, inplace = True)
    docks.to_csv("./coordinatesGeo.csv", index=False)

In [8]:
def createGeodata():
    df4 = pd.read_csv("./data/popular_journeys.csv", sep=",")
    top = df4.sort_values("pop_journeys_value", ascending=False)
    coord = pd.read_csv("./coordinates.csv")
    top10 = top.head(10)
    top10 = pd.merge(top10, coord, on=["station"])
    top10.rename(columns={"station":"start_station", "lat": "start_lat", "lon": "start_lon", "end_station_name": "station"}, inplace = True)
    top10 = pd.merge(top10, coord, on=["station"])
    top10.rename(columns={"station":"end_station","lat":"end_lat", "lon":"end_lon"}, inplace=True)
    top10.drop(["stationid_x", "stationid_y"],axis=1, inplace = True)
    top10
    #   "LINESTRING (-3.7038 40.4168,38 56)","LineString"
    top10["geometry"] = "LINESTRING (" + top10["start_lon"].astype(str) + " " + top10["start_lat"].astype(str) + "," + top10["end_lon"].astype(str) + " " + top10["end_lat"].astype(str) + ")"
    top10["name"] = top10["start_station"] + " to " + top10["end_station"]
    top10Geo = top10[["name", "geometry"]]
    top10Geo.to_csv("./data/top10Geo.csv", index=False)

In [9]:
def _create_map():
    df = pd.read_csv("./data/coordinatesGeo.csv")
    dfgeo = gpd.GeoDataFrame(
    df.loc[:, [c for c in df.columns if c != "geometry"]],
    geometry=gpd.GeoSeries.from_wkt(df["geometry"]),
    crs="epsg:3005")
    def map():
        return dfgeo.hvplot(geo=True, hover_cols='Name',tiles=True, xaxis=None, yaxis=None, color=ACCENT, responsive=True)
    return pn.Column(
        pn.pane.HoloViews(hv.DynamicMap(map), sizing_mode="stretch_both"),
                sizing_mode="stretch_both",
    )

In [10]:
ACCENT = "#1c1c1c"
template = FastGridTemplate(
    title="London TFL Bike Journeys Dashboard",
    row_height=55,
    prevent_collision=True,
    save_layout=True,
    accent_base_color=ACCENT,
    header_background=ACCENT,
    header_accent_base_color="Light Coral",
    theme=DarkTheme,
    theme_toggle=False
)

In [15]:
template.main[:6, 0:6] = _create_map()
template.main[:5, 6:10] = _create_barandlinewidget(data=df2, title="Trips by month", x="month", xlabel="Month")
template.main[5:6,6:10] = pn.pane.Markdown("# Total Trips: 76450245",style={"margin-top":"-30px"}, width=400, height=0)
template.main[6:10, 0:10] = _create_barandline(data=df3, title="Trips by time of day", x="time", y="trips", y2="mean", xlabel="Time of day", ylabel="Trips (in millions)", label="Trips", label2="Mean")
template.main[10:15, 5:10] = _create_barandlinewidget(data=df, title="Trips by year", x="year", xlabel="Year")
template.main[10:11 ,:5] = pn.pane.Markdown("# 🚴🚴🚴   Over 800 Docks   🚴🚴🚴",style={"margin-top": "-25px"}, width=400, height=20)
template.main[15:15, 0:10] = pn.Spacer()

In [16]:
template.servable()
template.show()
template.save("./dashboardFastGrid.html"    )

Launching server at http://localhost:43027



 unexpected attribute 'major_label' to LinearAxis, similar attributes are major_label_policy, major_label_standoff or major_label_text_font

 unexpected attribute 'major_label' to LinearAxis, similar attributes are major_label_policy, major_label_standoff or major_label_text_font

 unexpected attribute 'major_label' to LinearAxis, similar attributes are major_label_policy, major_label_standoff or major_label_text_font

 unexpected attribute 'major_label' to LinearAxis, similar attributes are major_label_policy, major_label_standoff or major_label_text_font
